In [ ]:
import sys
import os
import pandas as pd
import MetaTrader5 as mt5

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from services.mt5_data_service import MT5DataService
from services.indicator_service import IndicatorService

print("Libraries and services imported successfully.")

print("\nAttempting to initialize connection to MT5 terminal...")
if not mt5.initialize():
    print("FATAL: initialize() failed, error code =", mt5.last_error())
    quit()

print("SUCCESS: Connection to MT5 terminal established.")

In [ ]:
data_svc = MT5DataService()

SYMBOL = "EURUSD"
TIMEFRAME = 'H1'
START_DATE = "2020-01-01"

print(f"\n--- Fetching Primary Asset Data ({SYMBOL}) ---")
historical_df = data_svc.get_all_historical_data(symbol=SYMBOL, timeframe_str=TIMEFRAME, start_date=START_DATE)

if historical_df is not None:
    print(f"\nSUCCESS: {SYMBOL} data fetched successfully.")
    print(f"Total H1 Candles Fetched: {len(historical_df)}")
    print(f"Data Range: from {historical_df.index.min()} to {historical_df.index.max()}")
else:
    print("\nERROR: Failed to fetch data. Please check the logs.")

In [ ]:
if historical_df is not None and not historical_df.empty:
    
    indicator_svc = IndicatorService()
    
    print("\n--- Calculating the 'Ultimate' suite of indicators for EURUSD ---")
    
    features_df = indicator_svc.add_all_indicators(historical_df)
    
    if features_df is not None and not features_df.empty:
        print("\nSUCCESS: All features successfully calculated.")
        print(f"Final DataFrame shape: {features_df.shape}")
        print(f"Data Range: from {features_df.index.min()} to {features_df.index.max()}")
        print("\nData Head with all features:")
        display(features_df.head())
    else:
        print("\nERROR: Feature calculation resulted in an empty DataFrame.")
else:
    print("\nSkipping feature engineering due to data fetching errors in the previous cell.")
    features_df = None

In [ ]:
if 'features_df' in locals() and features_df is not None and not features_df.empty:
    
    data_dir = '../../data'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    output_filename = "eurusd_h1_features.csv"
    output_path = os.path.join(data_dir, output_filename)
    
    features_df.to_csv(output_path)

    print(f"\nSUCCESS: Processed H1 feature data for EURUSD saved to: '{output_path}'")
else:
    print("\nERROR: No valid data to save. Please check the output of the previous cells.")

if 'mt5' in locals() and mt5.terminal_info():
    mt5.shutdown()
    print("\nConnection to MT5 terminal has been shut down. Notebook complete.")
else:
    print("\nNotebook complete (MT5 connection was already closed or failed to initialize).")